In [1]:
# import libraries
import numpy as np # linear algebra
import pandas as pd 
import sqlite3
import math

import warnings 
warnings.filterwarnings("ignore")

import shap
import matplotlib.pyplot as plt
from catboost import Pool, CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from feature_engine.encoding import RareLabelEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
import ast

pd.set_option('display.max_rows', 1000)

In [2]:
df = pd.read_csv("cars_24_combined.csv").drop(['Unnamed: 0'], axis=1).drop_duplicates()
print(df.shape)
df.head()
df['Type'].unique()
df.head()

(8015, 9)


Car Name    Year  Distance  Owner    Fuel Location      Drive  \
0       Maruti S PRESSO  2022.0      3878      1  PETROL    HR-98     Manual   
1         Hyundai Xcent  2018.0     32041      1  PETROL    TN-22     Manual   
2           Tata Safari  2021.0     96339      1  DIESEL    TS-08  Automatic   
3  Maruti Vitara Brezza  2019.0     51718      1  DIESEL    WB-24     Manual   
4            Tata Tiago  2021.0     19811      1  PETROL    HR-51     Manual   

        Type    Price  
0  HatchBack   514000  
1      Sedan   674000  
2        SUV  1952000  
3        SUV   690000  
4  HatchBack   526000

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8015 entries, 0 to 8014
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Car Name  8014 non-null   object 
 1   Year      8014 non-null   float64
 2   Distance  8015 non-null   int64  
 3   Owner     8015 non-null   int64  
 4   Fuel      8015 non-null   object 
 5   Location  7802 non-null   object 
 6   Drive     8015 non-null   object 
 7   Type      8015 non-null   object 
 8   Price     8015 non-null   int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 626.2+ KB


In [4]:
main_label = 'Price [Lakh INR]'
df[main_label] = df['Price']*1e-5

df['log10_Distance'] = df['Distance'].apply(lambda x: 1/5*round(5*np.log10(1+x)))
# convert years to string
df['Year'] = df['Year'].fillna('None').astype(str)
# fill NaN values
df['Location'] = df['Location'].fillna('None')
df['Car Name'] = df['Car Name'].fillna('None')
# lowercase car names
df['Car Name'] = df['Car Name'].str.lower()


encoder2 = LabelEncoder()
for colum in [ 'Fuel']:
    df[colum] = encoder2.fit_transform(df[colum])

for col in ['Car Name', 'Location']:
    encoder1 = RareLabelEncoder(n_categories=1, max_n_categories=60, replace_with='Other', tol=20/df.shape[0])
    df[col] = encoder1.fit_transform(df[[col]])
# drop unused columns


df['Drive'] = df['Drive'].replace('Manual', '0')
df['Drive'] = df['Drive'].replace('Automatic', '1')
df['Drive'] = df['Drive'].astype('int64')
df['Fuel'] = df['Fuel'].astype('int64')

df['Type'] = df['Type'].replace('Sedan', '4')
df['Type'] = df['Type'].replace('HatchBack', '0')
df['Type'] = df['Type'].replace('Lux_SUV', '1')
df['Type'] = df['Type'].replace('Lux_sedan', '2')
df['Type'] = df['Type'].replace('SUV', '3')
df['Type'] = df['Type'].astype('int64')


cols2drop = ['Price', 'Distance']
df = df.drop(cols2drop, axis=1)
print(df.shape)
df.head()
df.info()

(8015, 9)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 8015 entries, 0 to 8014
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Car Name          8015 non-null   object 
 1   Year              8015 non-null   object 
 2   Owner             8015 non-null   int64  
 3   Fuel              8015 non-null   int64  
 4   Location          8015 non-null   object 
 5   Drive             8015 non-null   int64  
 6   Type              8015 non-null   int64  
 7   Price [Lakh INR]  8015 non-null   float64
 8   log10_Distance    8015 non-null   float64
dtypes: float64(2), int64(4), object(3)
memory usage: 626.2+ KB


In [5]:
unique_names = df['Car Name'].unique()
print(unique_names)

['maruti s presso' 'hyundai xcent' 'Other' 'maruti vitara brezza'
 'tata tiago' 'maruti swift' 'hyundai i20' 'renault kwid'
 'hyundai grand i10' 'maruti ignis' 'honda brio' 'hyundai elite i20'
 'honda city' 'maruti baleno' 'honda wr-v' 'honda amaze' 'maruti alto 800'
 'maruti celerio' 'ford ecosport' 'maruti ciaz' 'datsun redi go'
 'hyundai santro xing' 'ford freestyle' 'maruti dzire' 'maruti alto'
 'hyundai new santro' 'maruti alto k10' 'maruti swift dzire'
 'maruti wagon r 1.0' 'hyundai grand i10 nios' 'maruti celerio x'
 'mahindra xuv500' 'hyundai verna' 'hyundai venue' 'tata nexon'
 'toyota yaris' 'renault triber' 'renault duster' 'hyundai i10'
 'nissan magnite' 'maruti ertiga' 'honda jazz' 'kia seltos'
 'volkswagen ameo' 'renault kiger' 'hyundai new i20' 'tata altroz'
 'maruti ritz' 'hyundai eon' 'hyundai creta' 'toyota etios liva'
 'maruti new wagon-r' 'tata tigor' 'volkswagen polo'
 'toyota corolla altis' 'volkswagen vento' 'maruti s cross'
 'hyundai i20 active' 'hyundai aura' '

In [6]:
conn = sqlite3.connect("form_data.db")

# Query to retrieve all data from the form_data table
query = "SELECT * FROM form_data"

# Create a DataFrame using pandas
dft = pd.read_sql(query, conn)

# Close the database connection
conn.close()


# Display the DataFrame
print(dft.head())


   id                  name               email         car_name    year  \
0   1                 Aa_De  asdfsadf@gmail.com  maruti s presso  2022.0   
1   2  Aayuraditya Saraswat  Saraswat@gmail.com      maruti ciaz  2022.0   
2   3  Aayuraditya Saraswat  Saraswat@gmail.com      honda amaze  2014.0   
3   4                 Aa_De  asdfsadf@gmail.com   ford freestyle  2022.0   
4   5  Aayuraditya Saraswat  Saraswat@gmail.com    ford ecosport  2022.0   

  distance owner    fuel location   drive       type  
0    59999     1  PETROL    HR-26  Manual  HatchBack  
1    39999     1  PETROL    DL-07  Manual  HatchBack  
2   100000     1  PETROL    HR-06  Manual      Sedan  
3     9998     3  PETROL    DL-09  Manual      Sedan  
4     9998     1  PETROL    HR-06  Manual  HatchBack  


In [7]:

colsdrop = ['id', 'name', 'email']
dft = dft.drop(colsdrop, axis=1)


dft.rename(columns={'car_name': 'Car Name', 'year':'Year', 'owner': 'Owner', 'fuel': 'Fuel', 'location': 'Location', 'drive':'Drive', 'type': 'Type'}, inplace=True)

# dft['Distance'] = dft['Distance'].astype('int64')
dft['Owner'] = dft['Owner'].astype('int64')


# dft.drop('Distance', axis=1)
# print(dft.head())

dft['Type'] = dft['Type'].replace('Sedan', '4')
dft['Type'] = dft['Type'].replace('HatchBack', '0')
dft['Type'] = dft['Type'].replace('Lux_SUV', '1')
dft['Type'] = dft['Type'].replace('Lux_sedan', '2')
dft['Type'] = dft['Type'].replace('SUV', '3')

dft['distance'] = dft['distance'].astype('int64')
dft['log10_Distance'] = dft['distance'].apply(lambda x: 1/5*round(5*np.log10(1+x)))

cols1 = ['distance']
dft = dft.drop(cols1, axis=1)

dft['Fuel'] = dft['Fuel'].replace('PETROL', '3')
dft['Fuel'] = dft['Fuel'].replace('DIESEL', '1')
dft['Fuel'] = dft['Fuel'].replace('CNG', '0')
dft['Fuel'] = dft['Fuel'].replace('Other', '2')
dft['Fuel'] = dft['Fuel'].astype('int64')


dft['Drive'] = dft['Drive'].replace('Manual', '0')
dft['Drive'] = dft['Drive'].replace('Automatic', '1')
dft['Drive'] = dft['Drive'].astype('int64')


dft['Year'] = dft['Year'].astype('object')
dft['Type'] = dft['Type'].astype('int64')





dft.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Car Name        22 non-null     object 
 1   Year            22 non-null     object 
 2   Owner           22 non-null     int64  
 3   Fuel            22 non-null     int64  
 4   Location        22 non-null     object 
 5   Drive           22 non-null     int64  
 6   Type            22 non-null     int64  
 7   log10_Distance  22 non-null     float64
dtypes: float64(1), int64(4), object(3)
memory usage: 1.5+ KB


In [8]:
y = df[main_label].values.reshape(-1,)
X = df.drop([main_label], axis=1)
cat_cols = df.select_dtypes(include=['object']).columns
cat_cols_idx = [list(X.columns).index(c) for c in cat_cols]
X.shape,y.shape




X_test = dft.iloc[[-1]]

cat_colst = dft.select_dtypes(include=['object']).columns
cat_cols_idxt = [list(X_test.columns).index(c) for c in cat_colst]

# X.info()
# X_test.info()


In [9]:
train_pool = Pool(X, 
                  y, 
                  cat_features=cat_cols_idx)
test_pool = Pool(X_test,
                cat_features=cat_cols_idxt)


model = CatBoostRegressor(iterations=1000, 
                          depth=5,
                          verbose=0,
                          learning_rate=0.02, 
                          loss_function='RMSE')

model.fit(train_pool)

pred = model.predict(test_pool)

float_number = pred[0]

decimal_part = round(float_number % 1, 3)
decimal_part = decimal_part * 100
integer_part = math.floor(float_number)

d = str(decimal_part)
i = str(integer_part)

res_tup = "The predicted price of your car is", i, "Lakhs ",d, " Thousand" 

result = str(res_tup)

with open('result.txt', 'w') as file:
    file.write(result)